In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('./datasets/final_train_dataset.csv')

In [3]:
df.index = pd.to_datetime(df.Datetime)
new_df = df[['Average_Receive_bps', 'Average_Transmit_bps']]
new_df.head()

,Average_Receive_bps,Average_Transmit_bps
Datetime,,
2023-08-01 00:00:00,338950304.0,65747312.0
2023-08-01 00:05:00,338950304.0,65747312.0
2023-08-01 00:10:00,338873216.0,61819992.0
2023-08-01 00:15:00,338873216.0,61819992.0
2023-08-01 00:20:00,338956608.0,69735288.0


In [4]:
# Normalize the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(new_df)

# Define sequence length and features
sequence_length = 10  # Number of time steps in each sequence
num_features = len(new_df.columns)

# Create sequences and corresponding labels
sequences = []
labels = []
for i in range(len(scaled_data) - sequence_length):
    seq = scaled_data[i:i+sequence_length]
    label = scaled_data[i+sequence_length][1]  # Assuming 'average_receive_bps' is at index 1
    sequences.append(seq)
    labels.append(label)

# Convert to numpy arrays
sequences = np.array(sequences)
labels = np.array(labels)

# Split into train and test sets
train_x, test_x, train_y, test_y = train_test_split(sequences, labels, test_size=0.2, random_state=42)

In [5]:
def create_gru_model(units, learning_rate):
    model = Sequential()
    model.add(GRU(units, input_shape=(train_x.shape[1], train_x.shape[2]), return_sequences=False))
    model.add(Dense(1))  # Assuming regression task; change according to your use case
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])  # Modify metrics as needed
    
    return model

Define the Genetic Algorithm

In [6]:
from deap import base, creator, tools, algorithms
import random

# Create the fitness function and individual structure
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimize the loss
creator.create("Individual", list, fitness=creator.FitnessMin)

# Define hyperparameter ranges
def random_units():
    return random.randint(10, 200)  # Number of GRU units

def random_learning_rate():
    return random.uniform(0.0001, 0.01)  # Learning rate range

# Create the toolbox
toolbox = base.Toolbox()

# Register the hyperparameters to the toolbox
toolbox.register("units", random_units)
toolbox.register("learning_rate", random_learning_rate)

# Register individual and population
toolbox.register("individual", tools.initCycle, creator.Individual, (toolbox.units, toolbox.learning_rate), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Define the fitness function
def evaluate(individual):
    # Ensure units is a positive integer
    units = max(1, int(round(individual[0])))  # Ensure it's at least 1
    learning_rate = individual[1]

    # Create and compile the GRU model
    model = create_gru_model(units, learning_rate)

    # Train the model for a few epochs to test the configuration
    model.fit(train_x, train_y, epochs=5, batch_size=32, verbose=0, validation_split=0.1)

    # Evaluate the model on the test set
    loss, mae = model.evaluate(test_x, test_y, verbose=0)

    return loss,  # Return loss for minimization



toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

# Parameters
population_size = 10
num_generations = 20
prob_crossover = 0.7
prob_mutation = 0.2

# Create the population
population = toolbox.population(n=population_size)

# Run the GA optimization
algorithms.eaSimple(population, toolbox, cxpb=prob_crossover, mutpb=prob_mutation, ngen=num_generations, verbose=True)


C:\Users\Saint_Hacker\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


gen	nevals
0  	10    


KeyboardInterrupt: 

In [ ]:
# Get the best individual (set of hyperparameters)
best_individual = tools.selBest(population, k=1)[0]
best_units = best_individual[0]
best_learning_rate = best_individual[1]

print(f"Best units: {best_units}, Best learning rate: {best_learning_rate}")

# Train the final model with the best hyperparameters
final_model = create_gru_model(best_units, best_learning_rate)
final_model.fit(train_x, train_y, epochs=20, batch_size=32, validation_split=0.1)
